In [ ]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt

In [ ]:
!wget --no-check-certificate \
   'http://vis-www.cs.umass.edu/lfw/lfw.tgz'\
    -O "/tmp/lfw.tgz"
local_zip = "/tmp/lfw.tgz"
import os, sys, tarfile

def extract(tar_url, extract_path=''):
    tar = tarfile.open(tar_url, 'r')
    for item in tar:
        tar.extract(item, extract_path)
        if item.name.find(".tgz") != -1 or item.name.find(".tar") != -1:
            extract(item.name, "./" + item.name[:item.name.rfind('/')])

extract('/tmp/lfw.tgz')
print ('Done.')



--2022-05-07 09:51:28--  http://vis-www.cs.umass.edu/lfw/lfw.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180566744 (172M) [application/x-gzip]
Saving to: ‘/tmp/lfw.tgz’

/tmp/lfw.tgz        100%[===================>] 172.20M  32.0MB/s    in 5.9s    

2022-05-07 09:51:34 (29.0 MB/s) - ‘/tmp/lfw.tgz’ saved [180566744/180566744]

Done.


In [ ]:
source_path='/content/lfw'
i=0
for rootdir, dirs, files in os.walk(source_path):
    for subdir in dirs:
        i=i+1
        print(os.path.join(source_path, subdir))
print(i)

Streaming output truncated to the last 5000 lines.
/content/lfw/Strom_Thurmond
/content/lfw/Makiko_Tanaka
/content/lfw/Jon_Voight
/content/lfw/Ray_Young
/content/lfw/Ruben_Studdard
/content/lfw/John_Rowe
/content/lfw/Robert_Torricelli
/content/lfw/Fabian_Vargas
/content/lfw/Solomon_Passy
/content/lfw/Winona_Ryder
/content/lfw/Clint_Lamebear
/content/lfw/Jessica_Alba
/content/lfw/Maurice_Strong
/content/lfw/Eugene_Teslovic
/content/lfw/Krishna_Bhadur_Mahara
/content/lfw/Roger_Daltrey
/content/lfw/Anatoliy_Kinakh
/content/lfw/Bill_Sizemore
/content/lfw/Paul_Sarbanes
/content/lfw/Derek_Parra
/content/lfw/Wendy_Kennedy
/content/lfw/Chuanyun_Li
/content/lfw/Daniel_Scioli
/content/lfw/Mitchell_Potter
/content/lfw/Andy_Perez
/content/lfw/Mike_Leach
/content/lfw/Sterling_Hitchcock
/content/lfw/Angela_Merkel
/content/lfw/Charlie_Williams
/content/lfw/Mario_Jardel
/content/lfw/Tiago_Splitter
/content/lfw/Bixente_LIzarazu
/content/lfw/Kevin_Millwood
/content/lfw/Christian_Patino
/content/lfw/Patr

In [ ]:
number_person=3 # this the number of person that you want to make the model with if u want to make a model with only 1 person you will need to make some changes in the model i will manchin later 
root_dir = '/tmp/faces'
# Empty directory to prevent FileExistsError is the function is run several times
if os.path.exists(root_dir):
  shutil.rmtree(root_dir)
def create_train_test_dirs(root_path,number_person):
  ### START CODE HERE

  # HINT:
  # Use os.makedirs to create your directories with intermediate subdirectories


  path_training = os.path.join(root_path, "training")
  path_testing=os.path.join(root_path, "testing")

 
  os.makedirs(path_training)
  os.makedirs(path_testing)
  os.makedirs(os.path.join(path_training,"dont_know"))
  os.makedirs(os.path.join(path_testing,"dont_know"))
  for i in range(1,number_person+1):
   os.makedirs(os.path.join(path_training,str(i)))
   os.makedirs(os.path.join(path_testing,str(i)))

  

try:
  create_train_test_dirs(root_path=root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [ ]:
source_path= '/tmp/faces'
i=0
# this code just to test for the creation of folder in the previes block
for rootdir, dirs, files in os.walk(source_path):
    for subdir in dirs:
        i=i+1
        print(os.path.join(source_path, subdir))
print(i)

/tmp/faces/training
/tmp/faces/testing
/tmp/faces/dont_know
/tmp/faces/1
/tmp/faces/3
/tmp/faces/2
/tmp/faces/dont_know
/tmp/faces/1
/tmp/faces/3
/tmp/faces/2
10


In [ ]:
#!rm -rf /tmp/faces/trainig/.ipynb_checkpoints 
# you  run this code if you  test the creation of the folder and found this checkpoint file (incomment the first line )

In [ ]:
path_training = os.path.join('/tmp',"dont_know") # the folder dont_know will have photo of people that model don't know , this photo are from the .tar file 
os.makedirs(path_training)
def dont_know(SOURCE):
  l=os.listdir(SOURCE)
  for img in l:
       copyfile( os.path.join( SOURCE,img) , os.path.join('/tmp/dont_know',img))
       break

In [ ]:
source_path='/content/lfw'
for rootdir, dirs, files in os.walk(source_path):
    i=0
    for subdir in dirs:
        i=i+1
        dont_know(os.path.join(source_path, subdir))
        if i==1781: # i chose to get from the folder only 1781 photo and i will get only 1781 photo from each video too 
          break
        

In [ ]:
print(f"\n\nThere are {len(os.listdir('/tmp/dont_know/'))} images of peaple that are not user for training")



There are 1781 images of peaple that are not user for training


In [ ]:
# GRADED FUNCTION: split_data
# this fuction split data between training and testing with the split size 
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):

  ### START CODE HERE
   l=os.listdir(SOURCE)
   l=random.sample(l , len(l))
   l=random.sample(l,len(l))
   s=0
   k=0
   for img in l:
     if os.path.getsize(SOURCE+img)==0:
       print( img+" is zero length, so ignoring.")
       k=k+1
   t=int(SPLIT_SIZE*(len(l)-k))
   for img in l:
     if os.path.getsize(SOURCE+img)==0:
       continue
     else:
       s=s+1
       if s<=t:
         copyfile( os.path.join( SOURCE,img) , os.path.join(TRAINING,img))
       else : 
         copyfile( os.path.join( SOURCE,img), os.path.join(TESTING,img))

  ### END CODE HERE

In [ ]:
import os
path_training = os.path.join('/tmp',"video")
os.makedirs(path_training) # creation of this folder to use it later 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/video_face

/content/drive/MyDrive/video_face


In [ ]:
from datetime import timedelta
import cv2
import numpy as np
import os

In [ ]:
def get_photo_from_video(source,photo_path):

  vidcap = cv2.VideoCapture(source)
  vidcap.set(cv2.CAP_PROP_POS_MSEC,10)      # just cue to 20 sec. position
  success,image = vidcap.read()
  count = 0
  while success:
    cv2.imwrite(photo_path+'/'+"frame%d.jpg" % count, image)     # save frame as JPEG file      
    success,image = vidcap.read()
    print('Read a new frame: ', success)
    count += 1
    if count==1781:
      break
    # this fuction will capture each frame from the video and save it in the photo path 

In [ ]:
import cv2
def split_video(source):
  l=os.listdir(source)
  i=0
  for vid in l:
    i=i+1
    photo_path = os.path.join('/tmp/', str(i))
    print(i)
    os.makedirs(photo_path)
    get_photo_from_video(source+vid,photo_path)
    # this function will use the get_photo_from_video to each video and save the photo captured from each video to separet file 
    

In [ ]:
split_video("/content/drive/MyDrive/video_face/")

Streaming output truncated to the last 5000 lines.
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new fr

In [ ]:
print(f"There are {len(os.listdir('/tmp/2'))} images of person 2")

There are 1781 images of person 2


In [ ]:
print(f"There are {len(os.listdir('/tmp/1'))} images of person 1")

There are 1781 images of person 1


In [ ]:
print(f"There are {len(os.listdir('/tmp/3'))} images of person 3")

There are 1781 images of person 3


In [ ]:
def delete_file(source):
 if len(os.listdir(source)) > 0:
   for file in os.scandir(source):
     os.remove(file.path)

In [ ]:
def copy_file(source,destination):
   l=os.listdir(source)
   for img in l:
         copyfile( os.path.join( source,img) , os.path.join(destination,img))

In [ ]:
from PIL import Image
def flip_image(source):
  l=os.listdir(source)
  for img in l:
   input_image = Image.open(source+img)
  #rotate image 
   angle1 = 90
   output1 = input_image.rotate(angle1)
   cv2.imwrite('/tmp/video'+'/'+img , np.asanyarray(output1)) 


In [ ]:
flip_image('/tmp/2/')
delete_file('/tmp/2/')
copy_file('/tmp/video/','/tmp/2/')
delete_file('/tmp/video/')
# i just found that the second video have all the photo flip it 90° so i make all the other these function to flip the image and save it to other file than copy it to the source file that it came from 

In [ ]:
source_dont_know='/tmp/dont_know/'
training_dont_know='/tmp/faces/training/dont_know/'
testing_dont_know='/tmp/faces/testing/dont_know/'
source_client={}
training_client={}
testing_client={}
for i in range(1,4):
  source_client[i]='/tmp/'+str(i)+"/"
  training_client[i]='/tmp/faces/training/'+str(i)+"/"
  testing_client[i]='/tmp/faces/testing/'+str(i)+"/"
  

In [ ]:
split_size=0.85
split_data(source_dont_know, training_dont_know, testing_dont_know, split_size) # split the data from the dont_know file to testing and training 

In [ ]:
print(f"\n\nThere are {len(os.listdir(training_dont_know))} images of peaple that are not user for training")
print(f"There are {len(os.listdir(testing_dont_know))} images of peaple that are not user for training")



There are 1513 images of peaple that are not user for training
There are 268 images of peaple that are not user for training


In [ ]:

for i in range(1,number_person+1):
  split_data(source_client[i], training_client[i], testing_client[i], split_size)

In [ ]:
print(f"\n\nThere are {len(os.listdir(training_dont_know))} images of peaple that are not user for training")
print(f"There are {len(os.listdir(testing_dont_know))} images of peaple that are not user for training")
for i in range(1,number_person+1):
  print(f"\n\nThere are {len(os.listdir(training_client[i]))} images of user with id "+str(i)+" for training")
  print(f"There are {len(os.listdir(testing_client[i]))} images of of user with id "+str(i)+" for training")



There are 1513 images of peaple that are not user for training
There are 268 images of peaple that are not user for training


There are 1513 images of user with id 1 for training
There are 268 images of of user with id 1 for training


There are 1513 images of user with id 2 for training
There are 268 images of of user with id 2 for training


There are 1513 images of user with id 3 for training
There are 268 images of of user with id 3 for training


In [ ]:
image_folder_train = '/tmp/faces/training'
img_height, img_width = 250, 250  # size of images
num_classes = number_person+1
split_size=0.85
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
dataset = keras.preprocessing.image_dataset_from_directory(
    image_folder_train,
    seed=42,
    image_size=(img_height, img_width),
    label_mode='categorical',# if u have only 1 person you want to work in tour model u need change clas_mode='binary' 
    shuffle=True)

Found 4539 files belonging to 3 classes.


In [ ]:
batch_size = 32

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.7, 1),
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    image_folder_train,
    target_size=(img_height, img_width),
    seed=42,
    batch_size=batch_size,
    class_mode='categorical')# if u have only 1 person you want to work in tour model u need change clas_mode='binary' 

Found 6052 images belonging to 4 classes.


In [ ]:
validation_dir='/tmp/faces/testing'

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(directory=validation_dir,
                                                                batch_size=batch_size, 
                                                               seed=42,                                              
                                                               
                                                              class_mode='categorical',# if u have only 1 person you want to work in tour model u need change clas_mode='binary' 
                                                         target_size = (250, 250))

Found 1072 images belonging to 4 classes.


In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.preprocessing import image

In [ ]:
base_model = keras.applications.ResNet50(weights='imagenet',
                                         include_top=False,  # without dense part of the network
                                         input_shape=(img_height, img_width, 3))

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
# Set layers to non-trainable
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
global_avg_pooling = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(number_person+1, activation='softmax')(global_avg_pooling)

face_classifier = keras.models.Model(inputs=base_model.input,
                                     outputs=output,
                                     name='ResNet50')
face_classifier.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 250, 250, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 256, 256, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 125, 125, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [ ]:
# ModelCheckpoint to save model in case of interrupting the learning process
checkpoint = ModelCheckpoint("models/face_classifier.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only=True,
                             verbose=1)

# EarlyStopping to find best model with a large number of epochs
earlystop = EarlyStopping(monitor='val_loss',
                          restore_best_weights=True,
                          patience=3,  # number of epochs with no improvement after which training will be stopped
                          verbose=1)

callbacks = [earlystop, checkpoint]

In [ ]:
face_classifier.compile(loss='categorical_crossentropy',
                        optimizer=keras.optimizers.Adam(learning_rate=0.01),
                        metrics=['accuracy'])

In [ ]:
epochs = 50

In [ ]:
history = face_classifier.fit(
    train_generator,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=validation_generator)

face_classifier.save("models/face_classifier.h5")# saving the model 

Epoch 1/50
190/190 [==============================] - ETA: 0s - loss: 0.9294 - accuracy: 0.6173
Epoch 1: val_loss improved from inf to 0.41717, saving model to models/face_classifier.h5
190/190 [==============================] - 272s 1s/step - loss: 0.9294 - accuracy: 0.6173 - val_loss: 0.4172 - val_accuracy: 0.7864
Epoch 2/50
190/190 [==============================] - ETA: 0s - loss: 0.5050 - accuracy: 0.8154
Epoch 2: val_loss improved from 0.41717 to 0.35976, saving model to models/face_classifier.h5
190/190 [==============================] - 264s 1s/step - loss: 0.5050 - accuracy: 0.8154 - val_loss: 0.3598 - val_accuracy: 0.8433
Epoch 3/50
190/190 [==============================] - ETA: 0s - loss: 0.3964 - accuracy: 0.8622
Epoch 3: val_loss improved from 0.35976 to 0.33225, saving model to models/face_classifier.h5
190/190 [==============================] - 263s 1s/step - loss: 0.3964 - accuracy: 0.8622 - val_loss: 0.3323 - val_accuracy: 0.8713
Epoch 4/50
190/190 [==================

In [ ]:
face_classifier.save("/tmp/face_classifier.h5")


In [ ]:
face_classifier = tf.keras.models.load_model("models/face_classifier.h5")


In [ ]:
cd /content/ 

/content


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(250, 250))
  x = image.img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  index = face_classifier.predict(images, batch_size=10)
  i=0
  for x in index[0]:
    if i==3:
      print('it is the person don t know  : '+" "+str(round(float(x*100),4))+"%")
    else:
       print('it is the person '+str(i+1)+" : "+str(round(float(x*100),4))+"%")
    i=i+1
#uploid photo from testing or other photo to the same person that the model train on for testing 

Saving frame1038.jpg to frame1038 (9).jpg
it is the person 1 : 0.0017%
it is the person 2 : 99.9678%
it is the person 3 : 0.0%
it is the person don t know  :  0.0305%
